# Cleaning

This notebook goes over the cleaning process and results of cleaning the dataset based on the problems observed in the [Profiling_The_Dataset.ipynb](https://colab.research.google.com/drive/1tk30gvS2qUptfBQTvsF68EuFWKPbwWY_?usp=sharing) notebook. The sample 311 service dataset with ~5 million rows is used in this notebook.

Just like the notebook for profiling, this notebook will also be broken down into the following sections: **Uniformity, Accuracy, Inconsistency, Completeness, and Outlier**. Each section will show the solution that we have come up with to solve the problem posed in the corresponding section in the profiling notebook. 

## Mount Google Drive & Create Dataset Folder

****NOTE: THIS IS FOR GOOGLE COLAB!**
(If you are not running on colab, you should manually create the directories, upload the dataset, and set the **WORKING_DIRECTORY** and **datafile** paths) 

Before we start cleaning the dataset, we need to mount our Google Drive to this notebook. Run the following cells to mount Google Drive, create directories leading to the dataset, and define the path to dataset. 

(Make sure to run through these cells even if this has already been done during the profiling notebook!)

In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create evq_311_service_proj and evq_311_service_proj/dataset folders
import os

WORKING_DIRECTORY = "/content/drive/MyDrive/evq_311_service_proj"

# Create path if the path does not already exist
if not os.path.exists(WORKING_DIRECTORY):
  os.chdir('/content/drive/MyDrive/')
  os.mkdir('evq_311_service_proj')
  os.chdir('evq_311_service_proj')
  os.mkdir('dataset')
  

os.chdir(WORKING_DIRECTORY)  # Change directory to WORKING_DIRECTORY
os.getcwd()     # Display current working directory
                # (Note: this should display WORKING_DIRECTORY)

'/content/drive/MyDrive/evq_311_service_proj'

In [ ]:
# Define dataset path (NOTE: replace "erm2-nwe9_5M.csv.gz" with your dataset name)
datafile = "./dataset/erm2-nwe9_5M.csv.gz"

By the end of this section, the current directory should be the **WORKING_DIRECTORY** path and the **datafile** path should be set. The final thing left to do is to **upload the dataset** into the folder that is expected by the **datafile** path, if this has not already been done so.

## Set Up pyspark

Now that the dataset path is set up and the dataset is uploaded, we need to set up pyspark to begin cleaning our dataset. 

Run through the following cells to set up pyspark.

In [ ]:
# Run this cell if pyspark is not already installed
%pip install pyspark

     |████████████████████████████████| 281.3 MB 17 kB/s 
     |████████████████████████████████| 198 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4cf6e55840465c882370f178c577600b10e574926e719ff324954072c9dc2ffc
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
# Set up pyspark session
from pyspark.sql import SparkSession

spark = SparkSession \
            .builder \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .config("spark.executor.memory", "45g") \
            .config("spark.driver.memory", "45g") \
            .getOrCreate()

df = spark.read.csv(datafile, header=True)

****WARNING:** If you run into a java heap memory, configure the following lines in the cell above: 


*  .config("spark.executor.memory", "30g") \
*  .config("spark.driver.memory", "30g") \

Change the number infront of the g (ex:20g)

Changing this number could also change the amount of RAM needed to download the final file


In [ ]:
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import to_timestamp

# Type casting to expected types
df = df.withColumn("Unique Key",df["Unique Key"].cast(IntegerType()))
df = df.withColumn("Due Date",to_timestamp(df["Due Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Created Date", to_timestamp(df["Created Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Closed Date",to_timestamp(df["Closed Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Incident Zip",df["Incident Zip"].cast(IntegerType()))
df = df.withColumn("BBL",df["BBL"].cast(IntegerType()))
df = df.withColumn("X Coordinate (State Plane)",df["X Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Y Coordinate (State Plane)",df["Y Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Latitude",df["Latitude"].cast(DoubleType()))
df = df.withColumn("Longitude",df["Longitude"].cast(DoubleType()))
df = df.withColumn("Resolution Action Updated Date",to_timestamp(df["Resolution Action Updated Date"],"MM/dd/yyyy hh:mm:ss a"))

df.printSchema()

root
 |-- Unique Key: integer (nullable = true)
 |-- Created Date: timestamp (nullable = true)
 |-- Closed Date: timestamp (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: integer (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: timestamp (nullable = true)
 |-- Resolution Description: string (nullable = true)
 |-- Resolu

Now that pyspark is set up and the columns of the dataset are updated to types that we expect, we can start using pyspark to clean the dataset!

## I. Uniformity

As observed during profiling, there are several non-uniform casing problems in the values of columns of type string. The problem is narrowed to the following five columns: "Complaint Type", "Descriptor", "Location Type", "Street Name", and "City". To solve this problem, we write a function called **oneColUniformCasing** that takes in a column name (type string) and updates the values of the items in the column to the format where the first character in every word is uppercased. 

Run the following cell with the function definition.

In [ ]:
def oneColUniformCasing(colName, df):
  # List of distinct values in column 
  items = list(df.select(colName).distinct().toPandas()[colName]) 

  # Remove any None/Null values (don't need to fix them)
  if (None in items):
    items.remove(None)

  # Applying title function to each item in the column
  itemsTitled = [i.title() for i in items]

  # Replacing the column values to a titled version of them
  df = df.replace(items,itemsTitled,colName) 

  return df

The function is applied to the columns that are found to be non-uniform.

In [ ]:
# Apply oneColUniformCasing to "Complaint Type"
df = oneColUniformCasing("Complaint Type", df)

# Apply oneColUniformCasing to "Descriptor"
df = oneColUniformCasing("Descriptor", df)

# Apply oneColUniformCasing to "Street Name"
df = oneColUniformCasing("Street Name", df)

# Apply oneColUniformCasing to "City"
df = oneColUniformCasing("City", df)

Since "Location Type" column only has one nonuniform value found, this value is directly corrected.

In [ ]:
from pyspark.sql.functions import regexp_replace

# Only fix row with "RESIDENTIAL BUILDING" in "Location Type" column
df = df.withColumn('Location Type', regexp_replace('Location Type', 
                                              'RESIDENTIAL BUILDING', 
                                              'Residential Building'))

### Try some columns to see the improvement!

Below are a few cells that show some of the columns that had uniformity problems. We can observe by running these cells that the items of each column now contain uniform casing (i.e. every value is in the format where the first letter of each word is uppercased).

In [ ]:
# View column "Complaint Type" 
df.select("Complaint Type").distinct().collect()

[Row(Complaint Type='Traffic Signal Condition'),
 Row(Complaint Type="Misc. Comments' And 6049=60..."),
 Row(Complaint Type='Animal-Abuse'),
 Row(Complaint Type='Street Sweeping Complaint'),
 Row(Complaint Type='Dof Parking - Request Copy'),
 Row(Complaint Type='Tanning'),
 Row(Complaint Type='Comments'),
 Row(Complaint Type='Noise - Helicopter'),
 Row(Complaint Type='Dof Property - Update Account'),
 Row(Complaint Type='Oem Disabled Vehicle'),
 Row(Complaint Type='Broken Parking Meter'),
 Row(Complaint Type='Fire Alarm - New System'),
 Row(Complaint Type='Window Guard'),
 Row(Complaint Type='Broken Muni Meter'),
 Row(Complaint Type='Highway Condition'),
 Row(Complaint Type='Dsny Spillage'),
 Row(Complaint Type='Street Condition'),
 Row(Complaint Type='Hazardous Materials'),
 Row(Complaint Type='Illegal Dumping'),
 Row(Complaint Type='Vending'),
 Row(Complaint Type='Ferry Permit'),
 Row(Complaint Type='Advocate - Rpie'),
 Row(Complaint Type='Taxi Report'),
 Row(Complaint Type='Advocate

In [ ]:
# View column "Location Type"
df.select("Location Type").distinct().collect()

[Row(Location Type='Apartment'),
 Row(Location Type='House and Store'),
 Row(Location Type='Ferry'),
 Row(Location Type='Private School'),
 Row(Location Type='Other (Explain Below)'),
 Row(Location Type='Public Park/Garden'),
 Row(Location Type='Roadway'),
 Row(Location Type='Loft Residence'),
 Row(Location Type='Condo Unit'),
 Row(Location Type='Cafeteria'),
 Row(Location Type='House of Worship'),
 Row(Location Type='Store/Commercial'),
 Row(Location Type='1-3 Family Mixed Use Building'),
 Row(Location Type='Building (Non-Residential)'),
 Row(Location Type='Public Area'),
 Row(Location Type='Golf'),
 Row(Location Type='Home'),
 Row(Location Type='Grocery Store'),
 Row(Location Type='Street Vendor'),
 Row(Location Type='Cafeteria - Private School'),
 Row(Location Type='Tanning Salon'),
 Row(Location Type='Abandoned Building'),
 Row(Location Type='Cemetery'),
 Row(Location Type='Subway'),
 Row(Location Type='Street Fair Vendor'),
 Row(Location Type='Co-Op Unit'),
 Row(Location Type='Par

In [ ]:
# View column "City"
df.select("City").distinct().collect()

[Row(City='Whyoming'),
 Row(City='Corona'),
 Row(City='Oak Dale'),
 Row(City='Couberstort'),
 Row(City='Staten Isand'),
 Row(City='Harrisburg'),
 Row(City='Carol Place'),
 Row(City='East Elm Hurst'),
 Row(City='Harrison'),
 Row(City='Hemphstead'),
 Row(City='Long Sland'),
 Row(City='Ste'),
 Row(City='Cranford'),
 Row(City='Phoenix'),
 Row(City='Cherry Hill'),
 Row(City='Levittown'),
 Row(City='East Rockaway'),
 Row(City='Bohenia'),
 Row(City='Oyster Bay'),
 Row(City='East Rutherford'),
 Row(City='Baldwin'),
 Row(City='Saddle Brook'),
 Row(City='Hollywood'),
 Row(City='Newark Airport'),
 Row(City='Monroe'),
 Row(City='North Plain Field'),
 Row(City='Newbrugh'),
 Row(City='Queens'),
 Row(City='Brookhaven'),
 Row(City='Newyok'),
 Row(City='Elmunt'),
 Row(City='Savannah'),
 Row(City='Rockville Center'),
 Row(City='Greenwood Village'),
 Row(City='Lynbrook'),
 Row(City='Lindenhurst'),
 Row(City='Ca'),
 Row(City='Coral Springs'),
 Row(City='Rego Park'),
 Row(City='Bay Shore'),
 Row(City='Omah

## II. Accuracy

During profiling, we used the [uszips.csv](https://drive.google.com/file/d/1qd2cXgTx-h-hRd0C7z2s_U4O8VYLAXA7/view?usp=sharing) dataset to uncover several inaccuracies in the City column of our 311 service dataset. To solve this problem, we use the zipcodes in our dataset to match with those in the [uszips.csv](https://drive.google.com/file/d/1qd2cXgTx-h-hRd0C7z2s_U4O8VYLAXA7/view?usp=sharing) dataset so that we can find the actual city values to replace those that are inaccurate.

Before we start, make sure to download and upload the [uszips.csv](https://drive.google.com/file/d/1qd2cXgTx-h-hRd0C7z2s_U4O8VYLAXA7/view?usp=sharing) dataset into the same folder/directory as where our dataset resides (i.e. for this colab notebook environment, the current working directory path should be /content/drive/MyDrive/evq_311_service_proj and the datasets should all be placed in the dataset/ folder inside the current working directory).

In [ ]:
# Define path for US zip dataset
uszip_path = "./dataset/uszips.csv"

In [ ]:
# Read the US zip dataset
us = spark.read.csv(uszip_path, header=True)
us = us.withColumn("zip",us["zip"].cast(IntegerType()))
us = us.withColumn("lat",us["lat"].cast(DoubleType()))
us = us.withColumn("lng",us["lng"].cast(DoubleType()))

In [ ]:
# Display US zip dataset
us.show()

+---+--------+---------+-------------+--------+-----------+----+-----------+----------+-------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|zip|     lat|      lng|         city|state_id| state_name|zcta|parent_zcta|population|density|county_fips|  county_name|      county_weights|    county_names_all|     county_fips_all|           imprecise|            military|            timezone|
+---+--------+---------+-------------+--------+-----------+----+-----------+----------+-------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|601|18.18005|-66.75218|     Adjuntas|      PR|Puerto Rico|TRUE|       null|     17113|  102.7|      72001|     Adjuntas|"{""72001"": ""99...| ""72141"": ""0.5...|     Adjuntas|Utuado|         72001|72141|               FALSE|               FALSE|
|602|18.

Run the following cells to solve the inaccuracy problem.

In [ ]:
import pyspark.sql.functions as F

# Checked the zip codes and found the cities for all rows this way any mistakes on spellings, uniformity can be removed at once.
result = df.join(us,df["Incident Zip"] == us["zip"],"left").select(df["Unique Key"],df["Incident Zip"],us["city"].alias("us_city"),df["City"],us["zip"].alias("us_zip")) # Left join on the basis of zip codes
temp = result.select(result["Unique Key"],result["us_city"],result["City"],F.coalesce(result["us_city"], result["City"]).alias("Final City"))

In [ ]:
# We are adding the final city column back to the main dataset
temp2= temp.select(temp["Unique Key"].alias("temp_uk"),temp["Final City"])
df = df.join(temp2,df["Unique Key"] ==  temp2["temp_uk"],how="left")
df = df.drop("temp_uk")

### Run the following cell to see the improvement!



Run the code for profiling here so that we can use it to compare to cleaned results to see the improvement.

In [ ]:
# All distinct cities present in the dataset
temp_df_1 = df.select("City").distinct() 
list_of_cities = list(temp_df_1.toPandas()['City']) 

# All cities present in the US zip dataset
list_of_correct_cities = list(us.select("city").distinct().toPandas()['city']) 

In [ ]:
list_of_wrong_cities = []

for i in list_of_cities:
  if i not in list_of_correct_cities:
    list_of_wrong_cities.append(i)

# First 20 mispelled words
list_of_wrong_cities[:20] 

['Whyoming',
 'Oak Dale',
 'Couberstort',
 'Staten Isand',
 'Carol Place',
 'East Elm Hurst',
 'Hemphstead',
 'Long Sland',
 'Ste',
 'Bohenia',
 'Newark Airport',
 'North Plain Field',
 'Newbrugh',
 'Queens',
 'Newyok',
 'Elmunt',
 'Rockville Center',
 'Greenwood Village',
 'Ca',
 'Bx']

Run the cell below and observe that the number of mispellings decreased from 2897 to 1105. 

In [ ]:
# Show this as a comparison of how much we cleaned mispellings.
City_wrong = df.filter(df["City"].isin(list_of_wrong_cities))
Final_City_wrong = df.filter(df["Final City"].isin(list_of_wrong_cities))
print("The number of mispelled or non uniform cities in city column: " + str(City_wrong.count()))
print("The number of mispelled or non uniform cities in Final city column: " + str(Final_City_wrong.count()))

The number of mispelled or non uniform cities in city column: 2897
The number of mispelled or non uniform cities in Final city column: 1105


## III. Inconsistency

As seen during profiling, there is incorrect data mostly in the "Agency Name" Column, but we will need to clean both "Agency Name" and "Agency" Columns.

To go over changes in "Agency", we need to make specific changes. For example, there were formatting errors with "Mayorâ", and because both are specific cases, we will fix those manually. For the other errors that were shown in Agency, we can either filter out these errors, or we can check the "correct" dataset to see if the Agency Name has a match in the good dataset and change the Agency accordingly. However, our main focus will be on Agency Names because Agency Names has significantly more errors than Agency, and therefore, we will not be implementing these changes in this document at the moment.

"Agency Name" will be fixed by going through if there is a corresponding "Agency" in the same row that is also in the "correct" dataset. If there is a match, then we will change the value of the agency name to what it is supposed to be. There will be one type of Agency I will not be changing the name for: the DOE. This is because of the aformentioned "School" agency name problem where the name of the school that the DOE could be important. The 3-1-1 names will also go untouched for the same reason.

Run the following cells to fix "Mayorâ" problem.

In [ ]:
# Fix the Mayora Problem by finding "MAYORâ" and replacing it in Agency
from pyspark.sql.functions import when

df = df.withColumn("Agency", \
              when(df["Agency"][0:6] == "MAYORâ", "OSE").otherwise(df["Agency"]))

In [ ]:
# Fix the Mayora Problem by finding "Mayorâ" and replacing it in Agency Name
df = df.withColumn("Agency Name", \
              when(df["Agency Name"][0:6] == "Mayorâ", "Mayor's Special Office of Enforcement").otherwise(df["Agency Name"]))

Before moving on, make sure to download and upload the [NYC-Agency-Names.csv](https://drive.google.com/file/d/1EHpyXNOwCpv-NM0OTTqFBHeKikKpBtd1/view?usp=sharing) dataset into the same folder/directory as where our dataset resides (i.e. for this colab notebook environment, the current working directory path should be /content/drive/MyDrive/evq_311_service_proj and the datasets should all be placed in the dataset/ folder inside the current working directory).

This dataset is needed for fixing inconsistencies in "Agency Name" and "Agency". 

In [ ]:
# Define path for NYC Agency Names dataset
nycAgencyNames_path = "./dataset/NYC-Agency-Names.csv"

In [ ]:
# Read NYC Agency Names dataset 
agency_df = spark.read.csv(nycAgencyNames_path, header=True)

In [ ]:
# Show Schema for NYC Agency Names dataset
agency_df.printSchema()

root
 |-- Agency Acronym: string (nullable = true)
 |-- Agency: string (nullable = true)



Run the following cells to fix inconsistencies in "Agency Name" and "Agency".

In [ ]:
from pyspark.sql.functions import coalesce

# Checked the Agency Acronym and found the Agency Names for all rows this way any mistakes on spellings, uniformity can be removed at once.
result = df.join(agency_df,df["Agency"] ==  agency_df["Agency Acronym"],"left").select(df["Unique Key"],df["Agency"],agency_df["Agency"].alias("agency_name_df"),df["Agency Name"].alias("df_Agency"))

# We want to now turn the value of agency_name_df to null when df_Agency contains school
result = result.withColumn("agency_name_df", when( (result["Agency"]== "DOE")& (result["df_Agency"][0:6] == "School"), None).otherwise(result["agency_name_df"]) )
temp = result.select(result["Unique Key"].alias("Temp Key"),coalesce(result["agency_name_df"], result["df_Agency"]).alias("Final Agency"))

In [ ]:
# Update our Agency Name Column with the new Data
df = df.join(temp,df["Unique Key"] ==  temp["Temp Key"],how="left")
df = df.drop("Temp Key")
df = df.withColumn("Agency Name", df["Final Agency"])
df = df.drop("Final Agency")

### Run the following cells to see the improvements!



We can see that the "MAYORâ" is now replaced with "Mayor's".

In [ ]:
# Show that the "MAYORâ" problem is fixed
df.select("Agency", "Agency Name").filter(df["Agency"] == "OSE").show(20,False)

+------+-------------------------------------+
|Agency|Agency Name                          |
+------+-------------------------------------+
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor's Special Office of Enforcement|
|OSE   |Mayor

We can observe in the following result that the Agency Name looks more consistent and more accurate.



In [ ]:
from pyspark.sql.functions import asc

# Show new values for Agency Name
df.select("Agency Name").distinct().orderBy(asc("Agency Name")).show(df.count(), False)

+--------------------------------------------------------------------------------------------------+
|Agency Name                                                                                       |
+--------------------------------------------------------------------------------------------------+
|3-1-1                                                                                             |
|3-1-1 Call Center                                                                                 |
|311 Administrative Supervisor                                                                     |
|Administration for Children's Services                                                            |
|CEO                                                                                               |
|Department for the Aging                                                                          |
|Department of Buildings                                                                   

Run the following cell to further show the data set has improved by looking at incorrect data. We can observe that there is much less inconsistencies compared to what we saw during profiling (Note: we are not counting 3-1-1 and the variations of that as inconsistent).

In [ ]:
# All distinct agency name present in the dataset
df5 = df.select("Agency Name").distinct() 
list_of_agency_names = list(df5.toPandas()['Agency Name']) 

# All agency present in the dataset
list_of_correct_agency_names = list(agency_df.select("Agency").distinct().toPandas()['Agency']) 

In [ ]:
list_of_wrong_agency_names_final = []
for i in list_of_agency_names:
  if (i[0:6] != "School") and i not in list_of_correct_agency_names:
    list_of_wrong_agency_names_final.append(i)
for j in list_of_wrong_agency_names_final:
  print(j)

3-1-1
TAX
3-1-1 Call Center
311 Administrative Supervisor
MOC
CEO


## IV. Completeness

During profiling, we observed that several values in the City column are null (i.e. missing). One simple way to try and fix the city nulls is by using the "Borough" Column because the data set seems to use Borough names for city names. Therefore, we simply replace the null values that have a non null value for its Borough with the Borough name.

Manhattan is a slightly special case because New York = Manhattan in the city column. For this case we have to change the value of "MANHATTAN" in Borough to New York.

Any null value in "City" that also has a null "Borough" value will be marked as Unspecified.

In [ ]:
# Replace null values with "unspecified"
from pyspark.sql.functions import when, desc, initcap

df = df.withColumn("City", \
              when(df["City"].isNull() & (df["Borough"] == "MANHATTAN"), "New York").when(df["City"].isNull() &  df["Borough"].isNotNull() & (df["Borough"] != "MANHHATAN"), initcap(df["Borough"])).when(df["City"].isNull() &  df["Borough"].isNull(), "Unspecified").otherwise(df["City"]))

### Run the following cell to see the improvement!

Run the cell and see that there are no null values left in the City column.

In [ ]:
# Re-display number of values in City column
df.groupBy('City').count().orderBy(desc("count")).show(df.count(), False)

In [ ]:
# Count how many nulls left
df.filter(df["City"].isNull()).count()

0

## V. Outlier

As shown in our profiling, there are several outliers in three columns of timestamp type: "Closed Date", "Due Date", and "Resolution Action Updated Date". We want to filter out rows with dates that should not belong in the dataset (aka any date from the year before 2010 and any date after 2021). We want to filter the data out rather than try to fix it because there would be no way to find out the correct dates for these outliers. 

**NOTE:** There is a count between each filtering to show the amount the filter affects the dataset.

The results are also shown after each filtering. It can be observed that the date ranges are now reasonable (i.e. between 2010 and present/2021). 

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import min, max

In [ ]:
# Fixing dates from Closed Date
from pyspark.sql.functions import year, desc

df = df.filter(df["Closed Date"].isNull() | (year("Closed Date") >= 2010) & (year("Closed Date") <= 2021))

In [ ]:
# Display results
df.select(min("Closed Date"),max("Closed Date")).show(df.count(), False)

+-------------------+-------------------+
|min(Closed Date)   |max(Closed Date)   |
+-------------------+-------------------+
|2010-01-01 00:00:00|2021-11-13 12:00:00|
+-------------------+-------------------+



In [ ]:
# Count of the number of overall rows currently in the data
df.count()

5413405

In [ ]:
# Fixing dates from Due Dates
from pyspark.sql.functions import year, desc

df = df.filter(df["Due Date"].isNull() | (year("Due Date") >= 2010) & (year("Due Date") <= 2021))

In [ ]:
# Display results
df.select(min("Due Date"),max("Due Date")).show(df.count(), False)

+-------------------+-------------------+
|min(Due Date)      |max(Due Date)      |
+-------------------+-------------------+
|2010-01-01 08:15:08|2021-12-12 13:43:56|
+-------------------+-------------------+



In [ ]:
# Count of the number of overall rows currently in the data to check we didn't get rid of too many rows
df.count()

5413330

In [ ]:
# Fixing dates from Resolution Action Updated Date
from pyspark.sql.functions import year, desc

df = df.filter(df["Resolution Action Updated Date"].isNull() | (year("Resolution Action Updated Date") >= 2010) & (year("Resolution Action Updated Date") <= 2021))

In [ ]:
# Display results
df.select(min("Resolution Action Updated Date"),max("Resolution Action Updated Date")).show(df.count(), False)

+-----------------------------------+-----------------------------------+
|min(Resolution Action Updated Date)|max(Resolution Action Updated Date)|
+-----------------------------------+-----------------------------------+
|2010-01-01 00:00:00                |2021-11-13 12:00:00                |
+-----------------------------------+-----------------------------------+



In [ ]:
# Count of the number of overall rows currently in the data
df.count()

5413327

## Conclusion

In this notebook, we found solutions for majority of the problems that were discovered during the profiling of our sample 311 service dataset, in [Profiling_The_Dataset.ipynb](https://colab.research.google.com/drive/1tk30gvS2qUptfBQTvsF68EuFWKPbwWY_?usp=sharing). We were also able to observe improvement in the dataset for every cleaning step that we took. 

**NOTE:** You need to have ~35GB of RAM to download the new dataset.

# Download the new dataset

Run the following cell to download the new, cleaned dataset.

In [ ]:
#Cast Certain Columns into strings as it will otherwise mess up download RAM & Crash the notebook
from pyspark.sql.types import StringType
from pyspark.sql.functions import from_unixtime,unix_timestamp

# Type casting to string types
df = df.withColumn("Unique Key",df["Unique Key"].cast(StringType()))
df = df.withColumn("Due Date",from_unixtime(unix_timestamp(df["Due Date"]),"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Created Date", from_unixtime(unix_timestamp(df["Created Date"]),"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Closed Date",from_unixtime(unix_timestamp(df["Closed Date"]),"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Incident Zip",df["Incident Zip"].cast(IntegerType()))
df = df.withColumn("BBL",df["BBL"].cast(IntegerType()))
df = df.withColumn("X Coordinate (State Plane)",df["X Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Y Coordinate (State Plane)",df["Y Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Latitude",df["Latitude"].cast(StringType()))
df = df.withColumn("Longitude",df["Longitude"].cast(StringType()))
df = df.withColumn("Resolution Action Updated Date",from_unixtime(unix_timestamp(df["Resolution Action Updated Date"]),"MM/dd/yyyy hh:mm:ss a"))

df.printSchema()

root
 |-- Unique Key: string (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: integer (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: string (nullable = true)
 |-- Resolution Description: string (nullable = true)
 |-- Resolution Actio

In [ ]:
# Download the new dataset 
# (Replace "./erm2-nwe9_5M_cleaned.csv.gz" with whatever name you would like)
df.write.option("compression","gzip").option("header", "true").csv("./erm2-nwe9_5M_cleaned.csv.gz")